In [28]:
import os
import subprocess
import re
import pandas as pd
import shutil

encoding="cp932"
RootFolder="C:\\Users\\sugii syuji"

def searchFile(ROOT_PATH,saveFile):
    if not os.path.isdir(os.path.dirname(saveFile)):
        os.makedirs(os.path.dirname(saveFile))
    allFile={}
    count=0
    for pathname,dirnames,filenames in os.walk(ROOT_PATH):
        for filename in filenames:
            if filename.endswith(".java"):
                fullPath=os.path.join(pathname,filename)
                count=count+1
                if "package-info" not in fullPath:
                    parentFolder=re.sub(r"\\src\\.*","",fullPath)
                    if parentFolder in allFile:
                        allFile[parentFolder].append(fullPath)
                    else:
                        allFile.setdefault(parentFolder,[fullPath])
                        
    with open(saveFile,"w",encoding="utf-8") as f:              
        for key,item in allFile.items():
            for pathfile in item:
                print(pathfile,file=f)
            print(file=f)
    
def editClass(Spoonfile, CKfile, outPutFile):   
    df = pd.read_csv(CKfile,encoding=encoding)
    df = df[["file", "class","type", "dit", "fanin", "fanout","lcc","lcom*","loc","noc","rfc","innerClassesQty","totalFieldsQty","totalMethodsQty","wmc"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")
    spoon = pd.read_csv(Spoonfile,encoding=encoding)
    join = spoon.merge(CK, how="inner", on=["file", "class"])
    join["class"]=join["class"].str.replace("$",".",)
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )

def editMethod(SpoonFile, CKFile, outPutFile):
    df = pd.read_csv(CKFile,encoding=encoding)
    df = df[["file", "class", "method","fanin","fanout", "wmc", "loc","returnsQty","variablesQty","parametersQty","loopQty","anonymousClassesQty","innerClassesQty","line"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")    
    CK["method"]=CK["method"].str.replace(r"<[^>]*>", "", regex=True)

    spoon = pd.read_csv(SpoonFile,encoding=encoding)
    spoon["method"]=spoon["method"].str.replace("$",".")
    spoon["method"]=spoon["method"].str.replace(" ","")

    join = spoon.merge(CK, how="inner", on=["file", "class", "line"])
    join["class"]=join["class"].str.replace("$",".")
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )
  
def editFile(saveMetricsFile):
    CKClass=saveMetricsFile+"class.csv"
    CKMethod=saveMetricsFile+"method.csv"
    SpoonClass=saveMetricsFile+"spoonClass.csv"
    SpoonMethod=saveMetricsFile+"spoonMethod.csv"
    saveClass=saveMetricsFile+"joinClass.csv"
    saveMethod=saveMetricsFile+"joinMethod.csv"
    
    editClass(SpoonClass,CKClass,saveClass)
    editMethod(SpoonMethod,CKMethod,saveMethod)

def spoonAnalyze(textFile,saveFolder):
    SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",textFile,saveFolder]
    subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)

def ckAnalyze(gitProject,saveFolder):
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",gitProject,"False","0","True",saveFolder]
    subprocess.run(CKAnalyze,cwd=os.path.join(os.getcwd(),"ck_anonymous\\ck\\target"),check=True)
    
def gitCheckout(gitProject,version):
    checkout=["git","checkout",version]
    subprocess.run(checkout,cwd=gitProject,check=True)
    
def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass

def supportSmellNum(smellFileName,saveFile):
    truthDF=getCodeSmellClass()
    group=truthDF.groupby("nomeSistema")
    classes=group.get_group(smellFileName)
    print(smellFileName)
    df=pd.read_csv(saveFile+"joinClass.csv",encoding=encoding)
    print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
    data=(set(df['class']) & set(classes['class']))
    print(classes[~classes['class'].isin(df['class'])])
    

In [31]:
with open("version.txt","r") as f:
    versionString=f.read()
    
projects=versionString.split("\n")
for project in projects:
    tmpFactor=project.split(",")
    projectName=tmpFactor[0]
    versions=[tmpFactor[1],tmpFactor[2]]
    smellName=tmpFactor[3]
    
    gitProject=os.path.join(RootFolder,"gitProject",projectName)
    saveFolder=os.path.join(os.getcwd(),"data1",projectName)
    textFile=os.path.join(os.getcwd(),"data1",projectName,"full.txt")
    if os.path.isdir(saveFolder):
        shutil.rmtree(saveFolder)
    os.makedirs(saveFolder)
    
    i=1
    for version in versions:
        saveFile=os.path.join(saveFolder,str(i))
        gitCheckout(gitProject,version)
        searchFile(gitProject,textFile)
        spoonAnalyze(textFile,saveFile)
        ckAnalyze(gitProject,saveFile)
        editFile(saveFile)
        if i==1:
            supportSmellNum(smellName,saveFile)
        i=i+1


corenlp
320 / 320
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
dbeaver
116 / 116
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
guava
177 / 177
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
elasticsearch-analysis
2 / 2
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
hikari
3 / 3
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
javafaker
2 / 2
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
jedis
13 / 13
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
jenkins
69 / 69
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
jsoup
16 / 16
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
junit4
8 / 8
Empty DataFrame
Columns: [nomeSistema, class]
Index: []
libgdx
211 / 215
    nomeSistema  \
806      libgdx   
810      libgdx   
811      libgdx   
812      libgdx   

                                                                     class  
806                                     com.badlogic.gdx.backends.gwt.emu.  
810  c

In [33]:
with open("firstStep.txt","r") as f:
    stringProject=f.read()
folders=stringProject.split("\n")
for folder in folders:
    tmp=folder.split(",")
    folderName=tmp[0]
    df1=pd.read_csv("data1\\"+folderName+"\\1class.csv")
    df2=pd.read_csv("data1\\"+folderName+"\\1joinClass.csv")
    print(folderName)
    print("CKclass : ",len(df1))
    print("class : ",len(df2))
    print()

CoreNLP
CKclass :  4294
class :  4294

dbeaver
CKclass :  6511
class :  6510

guava
CKclass :  7074
class :  7104

analysis-ik
CKclass :  28
class :  28

HikariCP
CKclass :  159
class :  159

java-faker
CKclass :  201
class :  201

jedis
CKclass :  910
class :  910

jenkins
CKclass :  3896
class :  3892

jsoup
CKclass :  325
class :  325

junit4
CKclass :  1473
class :  1473

libgdx
CKclass :  3853
class :  3853

mockserver
CKclass :  993
class :  993

mybatis-3
CKclass :  1540
class :  1540

nanohttpd
CKclass :  140
class :  140

netty-socketio
CKclass :  155
class :  155

redisson
CKclass :  2063
class :  2064

retrofit
CKclass :  287
class :  287

shopizer
CKclass :  1318
class :  1318

spark
CKclass :  221
class :  221

webmagic
CKclass :  299
class :  299

xdm
CKclass :  305
class :  305

zookeeper
CKclass :  1603
class :  1603



In [ ]:
with open("firstStep.txt","r")as f:
    modifyString=f.read()
    
modifyFolders=modifyString.split("\n")
for modifyFolder in modifyFolders:
    tmpModify=modifyFolder.split(",")
    folder=tmpModify[0]
    i=1
    while i<=2:
        spoonFile="data1\\"+folder+"\\"+str(i)+"spoonMethod.csv"
        ckFile="data1\\"+folder+"\\"+str(i)+"method.csv"
        outPutFile="data1\\"+folder+"\\"+str(i)+"joinMethod.csv"
        editMethod(spoonFile,ckFile,outPutFile)
        i=i+1

In [ ]:
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)

df1=pd.read_csv("data2\\corenlpspoonMethod.csv")
df2=pd.read_csv("ck_anonymous\\corenlp2method.csv")
df1["flag"]=1
df2["flag"]=2
df1=df1[["file","class","method","line","flag"]]
df2=df2[["file","class","method","line","flag"]]
df2["class"]=df2["class"].str.replace("$Anonymous","$")
df1["method"]=df1["method"].str.replace("$",".")
df1["method"]=df1["method"].str.replace(" ","")
df2["method"]=df2["method"].str.replace(r"<[^>]*>", "", regex=True)
join=pd.concat([df1,df2])
#drop=join.drop_duplicates(keep=False,subset=["file","class","method"])
drop2=join.drop_duplicates(keep=False,subset=["file","class","line"])
#drop3=drop2
#drop3=drop3.drop_duplicates(keep=False,subset=["file","class","line"])
print("ck method数 : ",len(df2))
print("結合完了数 : ",int((len(join)-len(drop2))/2))
#display(drop)

In [34]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_colwidth",None)
pd.set_option("display.max_rows",None)
df1=pd.read_csv("data1\\jenkins\\1spoonClass.csv")
df2=pd.read_csv("data1\\jenkins\\1class.csv")
df1["flag"]=1
df2["flag"]=2
df1=df1[["file","class","flag"]]
df2=df2[["file","class","flag"]]
df2["class"]=df2["class"].str.replace("$Anonymous","$")
join=pd.concat([df1,df2])
drop=join.drop_duplicates(keep=False,subset=["file","class"])
print(len(df1),len(df2),len(join),len(drop))
print(len(drop[drop["flag"]==2]))
print("ckのクラス数 : ",len(df2))
print("結合後のクラス数 : ",int((len(join)-len(drop))/2))
display(drop[drop["flag"]==2].sort_values("class"))

3922 3896 7818 34
4
ckのクラス数 :  3896
結合後のクラス数 :  3892


,file,class,flag
1742,C:\Users\sugii syuji\gitProject\jenkins\core\src\main\java\hudson\model\Job.java,ChartLabel,2
991,C:\Users\sugii syuji\gitProject\jenkins\core\src\main\java\hudson\XmlFile.java,Eureka,2
178,C:\Users\sugii syuji\gitProject\jenkins\core\src\main\java\hudson\model\Job.java,FeedItem,2
918,C:\Users\sugii syuji\gitProject\jenkins\core\src\main\java\hudson\search\Search.java,Tag,2
